In [1]:
import  torch
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab ,read_category,process_file
import numpy as np
from torch import  optim

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [2]:
#设置数据目标 ，实际上val集为测试集
train_file = 'cnews.train.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

In [11]:
def train():
    model = TextRNN().cuda()
    #定义损失函数
    Loss = nn.MultiLabelSoftMarginLoss()  #本项目为分类任务，而且是多分类，且每个样本只能属于一个类的情形，适合MultiLabelSoftMarginLoss
    optimizer = optim.Adam(model.parameters(),lr=0.001) #设定优化器为Adam优化器


    best_val_acc = 0  #初始化最好准确率
    flag = 0                  #老师的训练方式是直接训练1000次，然后通过打擂法选出最优的模型，但是在做作业的时候发现首先数据集大，每次训练的时间特别长，其次是1000次，我估算了一下，4分钟一个epoch批次，1000次训练完得66个小时。。。所以这里自己设置了开关当
                              #当准确率达到一个标准时就不往下训练了，直接跳出循环。不知道这个算不算是早停法。
    for epoch in range(1000):
        print('epoch= ',epoch)
        if flag == 0:
            #分批训练
            for step ,(x_batch,y_batch) in enumerate(train_loader):
                x=x_batch.cuda() #将数据载入Gpu
                y=y_batch.cuda()
                out = model(x)
                loss = Loss(out,y)
                print('loss:',loss)
                #反向传播
                optimizer.zero_grad() #梯度清零
                loss.backward() #反向传播
                optimizer.step() #参数更新
                accuracy = np.mean((torch.argmax(out,1) == torch.argmax(y,1)).cpu().numpy()) #计算准确率
                print('accuracy',accuracy)

            #对模型进行验证
            if (epoch+1) % 5 == 0:
                for step ,(x_batch,y_batch) in enumerate(val_loader):
                    x = x_batch.cuda()
                    y = y_batch.cuda()
                    out = model(x)
                    accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
                    if accuracy > best_val_acc:
                        torch.save(model,'model.pkl')
                        best_val_acc = accuracy
                        print('model.pkl saved')
                        print('accuracy:',accuracy)
                        if accuracy > 0.8:
                            flag =1
        print('训练完毕！')



In [12]:
# 获取文本的类别及其对应id的字典，因为计算中需要数值类型计算，所以需要将文本转换成数值类型，这里cnews.vocab.txt相当于字典，将每个文本转换成id
categories, cat_to_id = read_category()
print(categories)
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
print(words)

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
['<PAD>', '，', '的', '。', '一', '是', '在', '0', '有', '不', '了', '中', '1', '人', '大', '、', '国', '', '2', '这', '上', '为', '个', '“', '”', '年', '学', '时', '我', '地', '和', '以', '到', '出', '来', '会', '行', '发', '：', '对', '们', '要', '生', '家', '他', '能', '也', '业', '金', '3', '成', '可', '分', '多', '现', '5', '就', '场', '新', '后', '于', '下', '日', '经', '市', '前', '过', '方', '得', '作', '月', '最', '开', '房', '》', '《', '高', '9', '8', '.', '而', '比', '公', '4', '说', ')', '将', '(', '都', '资', 'e', '6', '基', '用', '面', '产', '还', '自', '者', '本', '之', '美', '很', '同', '', '7', '部', '进', '但', '主', '外', '动', '机', '元', '理', '加', 'a', '全', '与', '实', '影', '好', '小', '间', '其', '天', '定', '表', '力', '如', '次', '合', '长', 'o', '体', '价', 'i', '所', '内', '子', '目', '电', '-', '当', '度', '品', '看', '期', '关', '更', 'n', '等', '工', '然', '斯', '重', '些', '球', '此', '里', '利', '相', '情', '投', '点', '没', '因', '已', '三', '心', '特', '明', '位', '两', '法', '从', '入', '名', '万', '手', '计', '性', '事', '只', '样', '示', 'r', '

In [13]:
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)  #根据\t进行分割，分离出标签和特征，并装入数组中。并根据字典将汉字转成id进行计算
print('x_train=', x_train)
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)


x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [14]:
import torch.utils.data as Data
#设置GPU
cuda = torch.device('cuda')
x_train, y_train = torch.LongTensor(x_train),torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val),torch.Tensor(y_val)

train_dataset =Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=128,shuffle=True)
val_dataset = Data.TensorDataset(x_val, y_val)
val_loader  = Data.DataLoader(dataset=val_dataset,batch_size=128)

train()


epoch=  0
loss: tensor(0.7340, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.140625
loss: tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0703125
loss: tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1171875
loss: tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss: tensor(0.7344, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss: tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.109375
loss: tensor(0.7346, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0859375
loss: tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1484375
loss: tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.1015625
loss: tensor(0.7341, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss: tensor(0.7349, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.125
loss: tensor(0.7343, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.0625
loss: tensor(0.7339,

accuracy 0.40625
loss: tensor(0.7184, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.328125
loss: tensor(0.7207, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.2890625
loss: tensor(0.7148, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7188, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3046875
loss: tensor(0.7151, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7200, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3203125
loss: tensor(0.7214, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.265625
loss: tensor(0.7164, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3203125
loss: tensor(0.7209, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3203125
loss: tensor(0.7202, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.265625
loss: tensor(0.7183, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.296875
loss: tensor(0.7166, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss:

loss: tensor(0.7194, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.34375
loss: tensor(0.7174, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3046875
loss: tensor(0.7137, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7107, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4375
loss: tensor(0.7160, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7150, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7200, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3203125
loss: tensor(0.7211, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.2890625
loss: tensor(0.7217, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.265625
loss: tensor(0.7141, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3984375
loss: tensor(0.7185, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.34375
loss: tensor(0.7139, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7159, device='c

loss: tensor(0.7216, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.2890625
loss: tensor(0.7234, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.28125
loss: tensor(0.7151, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7155, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.328125
loss: tensor(0.7196, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3046875
loss: tensor(0.7133, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7155, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3359375
loss: tensor(0.7158, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.34375
loss: tensor(0.7126, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7152, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7117, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7172, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3515625
loss: tensor(0.7157, device=

loss: tensor(0.7162, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7130, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7156, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.34375
loss: tensor(0.7070, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4296875
loss: tensor(0.7122, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3828125
loss: tensor(0.7129, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7103, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4140625
loss: tensor(0.7138, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7163, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3359375
loss: tensor(0.7097, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4453125
loss: tensor(0.7139, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.40625
loss: tensor(0.7088, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4140625
loss: tensor(0.7128, device

...

accuracy 0.3046875
loss: tensor(0.7118, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7136, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7131, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7084, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4140625
loss: tensor(0.7131, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.40625
loss: tensor(0.7117, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.40625
loss: tensor(0.7170, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3515625
loss: tensor(0.7137, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7140, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7177, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.28125
loss: tensor(0.7159, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3828125
loss: tensor(0.7135, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(

loss: tensor(0.7133, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.359375
loss: tensor(0.7122, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.40625
loss: tensor(0.7124, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3828125
loss: tensor(0.7144, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.328125
loss: tensor(0.7097, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3984375
loss: tensor(0.7118, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.375
loss: tensor(0.7106, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.40625
loss: tensor(0.7144, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3828125
loss: tensor(0.7095, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.4453125
loss: tensor(0.7146, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.3671875
loss: tensor(0.7127, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.390625
loss: tensor(0.7036, device='cuda:0', grad_fn=<MeanBackward0>)
accuracy 0.484375
loss: tensor(0.7126, device=

epoch=  281
训练完毕！
epoch=  282
训练完毕！
epoch=  283
训练完毕！
epoch=  284
训练完毕！
epoch=  285
训练完毕！
epoch=  286
训练完毕！
epoch=  287
训练完毕！
epoch=  288
训练完毕！
epoch=  289
训练完毕！
epoch=  290
训练完毕！
epoch=  291
训练完毕！
epoch=  292
训练完毕！
epoch=  293
训练完毕！
epoch=  294
训练完毕！
epoch=  295
训练完毕！
epoch=  296
训练完毕！
epoch=  297
训练完毕！
epoch=  298
训练完毕！
epoch=  299
训练完毕！
epoch=  300
训练完毕！
epoch=  301
训练完毕！
epoch=  302
训练完毕！
epoch=  303
训练完毕！
epoch=  304
训练完毕！
epoch=  305
训练完毕！
epoch=  306
训练完毕！
epoch=  307
训练完毕！
epoch=  308
训练完毕！
epoch=  309
训练完毕！
epoch=  310
训练完毕！
epoch=  311
训练完毕！
epoch=  312
训练完毕！
epoch=  313
训练完毕！
epoch=  314
训练完毕！
epoch=  315
训练完毕！
epoch=  316
训练完毕！
epoch=  317
训练完毕！
epoch=  318
训练完毕！
epoch=  319
训练完毕！
epoch=  320
训练完毕！
epoch=  321
训练完毕！
epoch=  322
训练完毕！
epoch=  323
训练完毕！
epoch=  324
训练完毕！
epoch=  325
训练完毕！
epoch=  326
训练完毕！
epoch=  327
训练完毕！
epoch=  328
训练完毕！
epoch=  329
训练完毕！
epoch=  330
训练完毕！
epoch=  331
训练完毕！
epoch=  332
训练完毕！
epoch=  333
训练完毕！
epoch=  334
训练完毕！
epoch=  335
训练完毕！
epoch=  33

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




训练完毕！
epoch=  736
训练完毕！
epoch=  737
训练完毕！
epoch=  738
训练完毕！
epoch=  739
训练完毕！
epoch=  740
训练完毕！
epoch=  741
训练完毕！
epoch=  742
训练完毕！
epoch=  743
训练完毕！
epoch=  744
训练完毕！
epoch=  745
训练完毕！
epoch=  746
训练完毕！
epoch=  747
训练完毕！
epoch=  748
训练完毕！
epoch=  749
训练完毕！
epoch=  750
训练完毕！
epoch=  751
训练完毕！
epoch=  752
训练完毕！
epoch=  753
训练完毕！
epoch=  754
训练完毕！
epoch=  755
训练完毕！
epoch=  756
训练完毕！
epoch=  757
训练完毕！
epoch=  758
训练完毕！
epoch=  759
训练完毕！
epoch=  760
训练完毕！
epoch=  761
训练完毕！
epoch=  762
训练完毕！
epoch=  763
训练完毕！
epoch=  764
训练完毕！
epoch=  765
训练完毕！
epoch=  766
训练完毕！
epoch=  767
训练完毕！
epoch=  768
训练完毕！
epoch=  769
训练完毕！
epoch=  770
训练完毕！
epoch=  771
训练完毕！
epoch=  772
训练完毕！
epoch=  773
训练完毕！
epoch=  774
训练完毕！
epoch=  775
训练完毕！
epoch=  776
训练完毕！
epoch=  777
训练完毕！
epoch=  778
训练完毕！
epoch=  779
训练完毕！
epoch=  780
训练完毕！
epoch=  781
训练完毕！
epoch=  782
训练完毕！
epoch=  783
训练完毕！
epoch=  784
训练完毕！
epoch=  785
训练完毕！
epoch=  786
训练完毕！
epoch=  787
训练完毕！
epoch=  788
训练完毕！
epoch=  789
训练完毕！
epoch=  790
训练完毕！
epo